In [1]:
from dependencies import *

userdata = pd.read_csv('data/yelp/user.csv', low_memory=False)
uid2num = {k:v for k,v in zip(userdata['user_id'].values,range(len(userdata['user_id'].values)))}
num2uid = {k:v for v,k in uid2num.items()}

udata = pickle.load(open( "udata.p", "rb" ))
adj = {uid2num[k]:[uid2num[x] for x in v.split(', ') if x in uid2num] for k,v in zip(udata['user_id'].values,udata['friends'].values)}


Using TensorFlow backend.


In [2]:
components = pickle.load(open( "graph_components_apr29.p", "rb" ))
#Get the training and the test connected component
gc = components[0]

def giveagraph(gc,start=0,end=10000):
	temp = nx.Graph()
	temp.add_nodes_from(list(gc.nodes)[start:end])
	temp.add_edges_from([(i,j) for i in temp.nodes for j in gc[i] if j in temp.nodes ])
	return max(nx.connected_component_subgraphs(temp), key=len)

#traing = giveagraph(gc)
#testg = giveagraph(gc,10000,20000)



In [3]:
def giveagraph_nodes(gc,nodes):
	temp = nx.Graph()
	temp.add_nodes_from(nodes)
	temp.add_edges_from([(i,j) for i in nodes for j in gc[i] if j in nodes ])
	return max(nx.connected_component_subgraphs(temp), key=len)

import random
nodes  = list(gc.nodes)[0:20000]
random.Random(4).shuffle(nodes)
traing = giveagraph_nodes(gc,nodes[:10000])
testg = giveagraph_nodes(gc,nodes[10000:])


In [4]:
len(testg.nodes)

7151

In [5]:
def sample_edges(g,possize=100,negsize=100,full = False):
    negexamples = list(nx.non_edges(g))
    totaledges = len(g.edges())
    if full:
        posset = np.array(g.edges())
        negsize = totaledges
    else:
        np.random.seed(4)
        posset = np.random.choice(np.array(g.edges()), possize)
    np.random.seed(4)
    negindices = np.random.choice(range(len(negexamples)), negsize)
    negset = np.array(negexamples)[negindices]
    return posset,negset

postrainset, negtrainset = sample_edges(traing, full = True)
postestset, negtestset = sample_edges(testg, full = True)

In [6]:
len(postestset)

71839

In [7]:
ids  = udata.to_numpy()[:,[0]]
features = udata.to_numpy()[:,[2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21]]
uid2feat = dict(zip(ids.reshape(-1),features))

In [8]:
uid2fset = {a:set(b.split(', ')) for a,b in zip(udata['user_id'].values,udata['friends'].values)}
uid2bidset = {a:b for a,b in zip(udata['user_id'].values,udata['bidset'].values)}
uid2cityset = {a:b for a,b in zip(udata['user_id'].values,udata['cityset'].values)}

def givenumcommon(uid2set,uid1,uid2):
    a = uid2set[uid1]
    b = uid2set[uid2]
    return len(a.intersection(b))

def givenumunion(uid2set,uid1,uid2):
    a = uid2set[uid1]
    b = uid2set[uid2]
    return len(a.union(b))

In [9]:
def givejaccard(uidset,uid1,uid2):
    a = givenumcommon(uidset,uid1,uid2)
    b = givenumunion(uidset,uid1,uid2)
    return np.round((a*1.0)/(b*1.0),2)

def giveprefattach(uid2set,uid1,uid2):
    a = uid2set[uid1]
    b = uid2set[uid2]
    return (len(a) * len(b))

In [10]:
def giveedgefeats(pairlist,cfriends=True,cbids=True,ccities= True):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        if cfriends:
            feats.append(givenumcommon(uid2fset,id0,id1))
        if cbids:
            feats.append(givenumcommon(uid2bidset,id0,id1))
        if ccities:
            feats.append(givenumcommon(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgefeats_jaccard(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        feats.append(givejaccard(uid2fset,id0,id1))
        feats.append(givejaccard(uid2bidset,id0,id1))
        feats.append(givejaccard(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgefeats_prefattach(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        feats.append(giveprefattach(uid2fset,id0,id1))
        feats.append(giveprefattach(uid2bidset,id0,id1))
        feats.append(giveprefattach(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgefeats_all(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        feats.append(givenumcommon(uid2fset,id0,id1))
        feats.append(givenumcommon(uid2bidset,id0,id1))
        feats.append(givenumcommon(uid2cityset,id0,id1))
        feats.append(givejaccard(uid2fset,id0,id1))
        feats.append(givejaccard(uid2bidset,id0,id1))
        feats.append(givejaccard(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgedata(posset,negset,cfriends=True,cbids=True,ccities= True):
    X1  = giveedgefeats(posset,cfriends,cbids,ccities)
    X2  = giveedgefeats(negset,cfriends,cbids,ccities)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def giveedgedata_jaccard(posset,negset):
    X1  = giveedgefeats_jaccard(posset)
    X2  = giveedgefeats_jaccard(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def giveedgedata_prefattach(posset,negset):
    X1  = giveedgefeats_prefattach(posset)
    X2  = giveedgefeats_prefattach(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


def giveedgedata_all(posset,negset):
    X1  = giveedgefeats_all(posset)
    X2  = giveedgefeats_all(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def givenodefeats(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        X.append(np.concatenate([uid2feat[id0], uid2feat[id1]]))
    return np.array(X)

def givenodedata(posset,negset):
    X1  = givenodefeats(posset)
    X2  = givenodefeats(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


def givedatausingbothfeats(posset,negset):
    X1  = np.concatenate((giveedgefeats(posset),givenodefeats(posset)),1)
    X2  = np.concatenate((giveedgefeats(negset),givenodefeats(negset)),1)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


def givedatausingbothfeats_jaccard(posset,negset):
    X1  = np.concatenate((giveedgefeats_jaccard(posset),givenodefeats(posset)),1)
    X2  = np.concatenate((giveedgefeats_jaccard(negset),givenodefeats(negset)),1)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def givefeats(pairlist):
    X = []
    for p in pairlist:
        id0 = p[0]
        id1 = p[1]
        X.append([id0,id1])
    return np.array(X)
    
def givedata(posset,negset):
    X1  = givefeats(posset)
    X2  = givefeats(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

In [11]:
def giveresults(Ytest,ytestpred):
    print("Accuracy ",np.round(accuracy_score(Ytest,ytestpred),3))
    print("Precision ",np.round(precision_score(Ytest,ytestpred),3))
    print("Recall ",np.round(recall_score(Ytest,ytestpred),3))
    print("F1 Score ",np.round(f1_score(Ytest,ytestpred),3))

In [12]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [13]:
from keras.models import Sequential
from keras.layers import Dense

from numpy.random import seed
seed(3)
from tensorflow import set_random_seed
set_random_seed(3)

In [14]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
from torch.autograd import Variable
import random
random.seed(3)

In [15]:
class Encoder(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, features, feature_dim, embed_dim, adj_lists, aggregator, n2mat, num_sample=10, base_model=None): 
        super(Encoder, self).__init__()
        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model
        self.embed_dim = embed_dim
        self.weight = nn.Parameter(torch.FloatTensor(embed_dim, 2 * self.feat_dim))
        init.xavier_uniform_(self.weight)
        self.n2mat = n2mat
    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes
        """
        #print(self)
        neigh_feats = self.aggregator.forward(nodes, [set(self.adj_lists[int(node)]) for node in nodes], self.num_sample)
        #self_feats = self.features(torch.LongTensor([self.n2mat[x] for x in nodes]))
        self_feats = self.features[[self.n2mat[x] for x in nodes]]
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        combined = F.relu(self.weight.mm(combined.t()))
        return combined

In [16]:
class MeanAggregator(nn.Module):
    """
    Aggregates a node's embeddings using mean of neighbors' embeddings
    """
    def __init__(self, features,n2mat): 
        super(MeanAggregator, self).__init__()
        self.features = features
        self.n2mat = n2mat
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        #print(self)
        _set = set
        samp_neighs = to_neighs    
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        #embed_matrix = self.features(torch.LongTensor([self.n2mat[x] for x in unique_nodes_list]))
        embed_matrix = self.features[[self.n2mat[x] for x in unique_nodes_list]]
        to_feats = mask.mm(embed_matrix)
        return to_feats

In [17]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.fc1 = nn.Linear(2*enc.embed_dim,24)
        self.fc2 = nn.Linear(24,16)
        self.fc3 = nn.Linear(16,2)
    def forward(self, nodes,test=False):
        self.enc = updateenc(self.enc,test)
        embeds1 = self.enc(nodes[:,0])
        embeds2 = self.enc(nodes[:,1])
        embeds = torch.cat([embeds1, embeds2], dim=0)
        hidden1 = F.relu(self.fc1(embeds.t()))
        hidden2 = F.relu(self.fc2(hidden1))
        return self.fc3(hidden2)

In [18]:
def updateenc(enc,test=False):
    enc.features = enc1(nodes).t()
    enc.aggregator.features = enc1(nodes).t()
    return enc

In [19]:
ttg = nx.Graph()
ttg.add_edges_from(list(traing.edges(data=True))+list(testg.edges(data=True)))
ttg.add_nodes_from(list(traing.nodes(data=True))+list(testg.nodes(data=True)))
feat_data = np.array([uid2feat[num2uid[x]] for x in ttg]).astype(dtype = 'float32')
nodes = list(ttg.nodes())
testnodes = list(testg.nodes())
trainnodes = list(traing.nodes())
n2mat = {i:j for j,i in enumerate(ttg.nodes)}
mat2n = {i:j for j,i in n2mat.items()}
features = torch.FloatTensor(feat_data)

In [20]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 15, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 15, 15, ttg, agg2,n2mat, base_model=enc1)

In [21]:
graphsage = SupervisedGraphSage(enc2)
x,y = givedata(postrainset,negtrainset)
xtest,ytest = givedata(postestset,negtestset)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

0 tensor(86.3487)
5 tensor(30.0581)
10 tensor(13.5879)
15 tensor(7.4119)
20 tensor(3.1574)
25 tensor(2.6200)
30 tensor(2.6951)
35 tensor(1.9687)
40 tensor(3.9672)
45 tensor(1.9114)
50 tensor(1.4784)
55 tensor(1.6750)
60 tensor(3.2879)
65 tensor(1.2400)
70 tensor(1.3324)
75 tensor(1.5246)
80 tensor(0.8865)
85 tensor(1.0063)
90 tensor(1.9248)
95 tensor(0.8662)
100 tensor(1.2226)
105 tensor(1.0177)
110 tensor(1.3139)
115 tensor(1.2134)
120 tensor(1.3767)
125 tensor(0.7284)
130 tensor(1.1222)
135 tensor(0.7570)
140 tensor(1.5308)
145 tensor(0.8694)
150 tensor(1.5971)
155 tensor(0.7454)
160 tensor(0.8079)
165 tensor(0.8381)
170 tensor(1.7374)
175 tensor(0.7759)
180 tensor(0.9990)
185 tensor(0.8890)
190 tensor(1.0059)
195 tensor(1.0471)


In [22]:
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

Accuracy  0.798
Precision  0.82
Recall  0.785
F1 Score  0.802


In [25]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 16, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 16, 16, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(15.7374)
5 tensor(3.0723)
10 tensor(4.5044)
15 tensor(1.8453)
20 tensor(1.3481)
25 tensor(0.9711)
30 tensor(1.0735)
35 tensor(0.8853)
40 tensor(0.6843)
45 tensor(0.7821)
50 tensor(0.5955)
55 tensor(1.4429)
60 tensor(0.6454)
65 tensor(0.5792)
70 tensor(0.8554)
75 tensor(0.5048)
80 tensor(0.5289)
85 tensor(0.4728)
90 tensor(0.4753)
95 tensor(0.7044)
100 tensor(0.8138)
105 tensor(0.5685)
110 tensor(1.4891)
115 tensor(0.5563)
120 tensor(1.9567)
125 tensor(0.5070)
130 tensor(0.4967)
135 tensor(0.5194)
140 tensor(0.5541)
145 tensor(0.4319)
150 tensor(0.4304)
155 tensor(1.3842)
160 tensor(0.3311)
165 tensor(0.3795)
170 tensor(0.4387)
175 tensor(0.3771)
180 tensor(0.3511)
185 tensor(0.4045)
190 tensor(0.4226)
195 tensor(0.4796)
Accuracy  0.801
Precision  0.806
Recall  0.798
F1 Score  0.802


In [26]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 17, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 17, 17, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(23.3287)
5 tensor(13.8694)
10 tensor(3.7746)
15 tensor(1.9770)
20 tensor(3.3500)
25 tensor(3.6127)
30 tensor(1.6593)
35 tensor(1.4157)
40 tensor(3.0206)
45 tensor(1.4091)
50 tensor(1.7368)
55 tensor(1.5746)
60 tensor(1.2395)
65 tensor(2.0416)
70 tensor(1.8512)
75 tensor(1.1583)
80 tensor(1.6145)
85 tensor(1.3043)
90 tensor(2.1656)
95 tensor(1.4310)
100 tensor(1.2941)
105 tensor(0.6223)
110 tensor(1.9811)
115 tensor(1.1620)
120 tensor(0.6461)
125 tensor(0.5776)
130 tensor(0.6812)
135 tensor(0.4866)
140 tensor(0.8487)
145 tensor(1.2366)
150 tensor(1.1712)
155 tensor(0.7333)
160 tensor(1.1156)
165 tensor(0.8406)
170 tensor(0.7784)
175 tensor(0.5833)
180 tensor(0.5927)
185 tensor(0.8451)
190 tensor(0.7322)
195 tensor(1.4686)
Accuracy  0.771
Precision  0.659
Recall  0.85
F1 Score  0.743


In [27]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 18, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 18, 18, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(22.3008)
5 tensor(5.8400)
10 tensor(3.7118)
15 tensor(2.3068)
20 tensor(2.8552)
25 tensor(1.7214)
30 tensor(1.5696)
35 tensor(2.5285)
40 tensor(1.4434)
45 tensor(1.2077)
50 tensor(0.8592)
55 tensor(2.2622)
60 tensor(1.2286)
65 tensor(1.8804)
70 tensor(1.4293)
75 tensor(1.1950)
80 tensor(0.9521)
85 tensor(1.5891)
90 tensor(0.6407)
95 tensor(1.2950)
100 tensor(1.0272)
105 tensor(1.3650)
110 tensor(0.7259)
115 tensor(0.6592)
120 tensor(0.8545)
125 tensor(0.8853)
130 tensor(0.9143)
135 tensor(0.5148)
140 tensor(0.5824)
145 tensor(0.4627)
150 tensor(0.6248)
155 tensor(0.8397)
160 tensor(0.6271)
165 tensor(0.7166)
170 tensor(0.5127)
175 tensor(0.7075)
180 tensor(1.0024)
185 tensor(0.7469)
190 tensor(0.9249)
195 tensor(0.5778)
Accuracy  0.789
Precision  0.717
Recall  0.837
F1 Score  0.772


In [30]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 14, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 14, 14, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(131.9749)
5 tensor(21.5338)
10 tensor(12.5915)
15 tensor(12.8721)
20 tensor(5.1118)
25 tensor(4.2291)
30 tensor(5.0770)
35 tensor(2.1741)
40 tensor(1.7479)
45 tensor(1.3906)
50 tensor(2.4856)
55 tensor(3.8061)
60 tensor(1.4532)
65 tensor(1.0547)
70 tensor(1.5311)
75 tensor(2.6240)
80 tensor(1.7206)
85 tensor(1.8064)
90 tensor(2.2836)
95 tensor(3.2060)
100 tensor(1.2405)
105 tensor(0.2966)
110 tensor(1.3272)
115 tensor(0.9778)
120 tensor(0.9812)
125 tensor(1.1760)
130 tensor(1.0163)
135 tensor(1.1990)
140 tensor(4.2620)
145 tensor(1.7355)
150 tensor(1.6201)
155 tensor(1.3860)
160 tensor(0.9423)
165 tensor(1.0385)
170 tensor(1.1260)
175 tensor(0.8766)
180 tensor(1.3247)
185 tensor(0.8330)
190 tensor(1.7259)
195 tensor(1.8095)
Accuracy  0.773
Precision  0.728
Recall  0.8
F1 Score  0.762


In [31]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 13, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 13, 13, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(55.3798)
5 tensor(36.0598)
10 tensor(19.1607)
15 tensor(9.7822)
20 tensor(9.4688)
25 tensor(6.9039)
30 tensor(5.4749)
35 tensor(2.6725)
40 tensor(3.6655)
45 tensor(2.6457)
50 tensor(2.2779)
55 tensor(2.3858)
60 tensor(1.6467)
65 tensor(1.0703)
70 tensor(1.8564)
75 tensor(1.9564)
80 tensor(2.2379)
85 tensor(1.5368)
90 tensor(2.2814)
95 tensor(1.2444)
100 tensor(1.0745)
105 tensor(1.8506)
110 tensor(2.5439)
115 tensor(1.5598)
120 tensor(0.7249)
125 tensor(1.1119)
130 tensor(0.7100)
135 tensor(0.5431)
140 tensor(2.5515)
145 tensor(0.9427)
150 tensor(1.5771)
155 tensor(1.1630)
160 tensor(1.0455)
165 tensor(0.7609)
170 tensor(1.1321)
175 tensor(0.9875)
180 tensor(0.8095)
185 tensor(0.5528)
190 tensor(0.7781)
195 tensor(0.8552)
Accuracy  0.757
Precision  0.873
Recall  0.708
F1 Score  0.782


In [32]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 12, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 12, 12, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(19.5582)
5 tensor(13.2077)
10 tensor(5.8570)
15 tensor(4.2053)
20 tensor(2.8801)
25 tensor(2.7962)
30 tensor(2.7676)
35 tensor(3.6003)
40 tensor(2.1711)
45 tensor(2.3089)
50 tensor(2.1443)
55 tensor(3.2787)
60 tensor(1.9635)
65 tensor(1.8828)
70 tensor(0.9320)
75 tensor(1.2480)
80 tensor(1.0462)
85 tensor(0.7324)
90 tensor(2.7540)
95 tensor(1.0083)
100 tensor(1.0115)
105 tensor(1.7019)
110 tensor(0.7791)
115 tensor(1.1974)
120 tensor(1.9402)
125 tensor(1.0576)
130 tensor(1.2715)
135 tensor(1.1942)
140 tensor(1.0393)
145 tensor(0.6401)
150 tensor(1.7123)
155 tensor(0.9950)
160 tensor(0.7063)
165 tensor(1.0313)
170 tensor(1.0632)
175 tensor(0.4776)
180 tensor(0.9114)
185 tensor(0.8035)
190 tensor(0.7696)
195 tensor(0.9471)
Accuracy  0.755
Precision  0.856
Recall  0.712
F1 Score  0.777


In [33]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 11, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 11, 11, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(60.2128)
5 tensor(37.6560)
10 tensor(9.4221)
15 tensor(5.4270)
20 tensor(3.6977)
25 tensor(5.7124)
30 tensor(2.8556)
35 tensor(2.8205)
40 tensor(3.9305)
45 tensor(2.2133)
50 tensor(3.2538)
55 tensor(2.7337)
60 tensor(3.6590)
65 tensor(1.7883)
70 tensor(1.5695)
75 tensor(2.0047)
80 tensor(1.0618)
85 tensor(1.7353)
90 tensor(1.8304)
95 tensor(1.5654)
100 tensor(1.2039)
105 tensor(0.7455)
110 tensor(1.9424)
115 tensor(1.6964)
120 tensor(1.2944)
125 tensor(1.6218)
130 tensor(0.8732)
135 tensor(1.3432)
140 tensor(1.2868)
145 tensor(2.0025)
150 tensor(1.5719)
155 tensor(0.9732)
160 tensor(0.9870)
165 tensor(1.5042)
170 tensor(0.9378)
175 tensor(1.4713)
180 tensor(1.2767)
185 tensor(0.6340)
190 tensor(0.4814)
195 tensor(1.0423)
Accuracy  0.757
Precision  0.678
Recall  0.805
F1 Score  0.736


In [34]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 19, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 19, 19, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(6.1066)
5 tensor(2.6316)
10 tensor(1.1648)
15 tensor(1.2193)
20 tensor(0.9194)
25 tensor(0.3824)
30 tensor(0.6241)
35 tensor(0.6781)
40 tensor(0.6215)
45 tensor(0.4889)
50 tensor(0.7198)
55 tensor(0.7817)
60 tensor(0.5965)
65 tensor(0.5112)
70 tensor(0.4656)
75 tensor(0.3228)
80 tensor(0.5370)
85 tensor(0.4534)
90 tensor(0.5918)
95 tensor(0.5103)
100 tensor(0.4160)
105 tensor(0.4762)
110 tensor(0.4587)
115 tensor(0.4674)
120 tensor(0.5009)
125 tensor(0.5701)
130 tensor(0.4908)
135 tensor(0.4161)
140 tensor(0.4078)
145 tensor(0.4950)
150 tensor(0.3838)
155 tensor(0.3800)
160 tensor(0.3538)
165 tensor(0.6167)
170 tensor(0.3738)
175 tensor(0.4238)
180 tensor(0.4169)
185 tensor(0.4096)
190 tensor(0.3117)
195 tensor(0.2767)
Accuracy  0.827
Precision  0.829
Recall  0.826
F1 Score  0.827


In [35]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 20, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 20, 20, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(57.5669)
5 tensor(22.6446)
10 tensor(12.4797)
15 tensor(6.3263)
20 tensor(4.9281)
25 tensor(0.7742)
30 tensor(2.2404)
35 tensor(2.2677)
40 tensor(1.5048)
45 tensor(0.8031)
50 tensor(1.3173)
55 tensor(0.9807)
60 tensor(0.7349)
65 tensor(1.0187)
70 tensor(0.6818)
75 tensor(0.9619)
80 tensor(1.5718)
85 tensor(0.7884)
90 tensor(1.6423)
95 tensor(0.7299)
100 tensor(0.4386)
105 tensor(0.8431)
110 tensor(0.7416)
115 tensor(0.6553)
120 tensor(0.7065)
125 tensor(0.6272)
130 tensor(0.5377)
135 tensor(0.6709)
140 tensor(0.9400)
145 tensor(0.4501)
150 tensor(0.5273)
155 tensor(0.5573)
160 tensor(0.3784)
165 tensor(0.6340)
170 tensor(0.5781)
175 tensor(0.5664)
180 tensor(1.3726)
185 tensor(0.4536)
190 tensor(0.6067)
195 tensor(0.4592)
Accuracy  0.789
Precision  0.828
Recall  0.768
F1 Score  0.797


In [36]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 50, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 50, 50, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(14.4615)
5 tensor(3.8853)
10 tensor(2.2624)
15 tensor(1.9057)
20 tensor(1.8163)
25 tensor(3.5430)
30 tensor(1.6745)
35 tensor(1.6784)
40 tensor(1.4359)
45 tensor(1.1801)
50 tensor(1.0388)
55 tensor(1.0197)
60 tensor(2.5836)
65 tensor(0.8260)
70 tensor(0.6559)
75 tensor(1.3281)
80 tensor(1.7722)
85 tensor(0.6260)
90 tensor(0.9001)
95 tensor(1.1291)
100 tensor(0.7057)
105 tensor(0.4740)
110 tensor(0.6175)
115 tensor(1.0605)
120 tensor(0.9118)
125 tensor(1.1030)
130 tensor(0.9446)
135 tensor(1.4943)
140 tensor(0.5717)
145 tensor(0.8606)
150 tensor(1.6501)
155 tensor(0.8220)
160 tensor(0.9668)
165 tensor(0.5026)
170 tensor(1.0481)
175 tensor(0.9604)
180 tensor(0.7535)
185 tensor(0.4976)
190 tensor(0.4716)
195 tensor(0.5703)
Accuracy  0.774
Precision  0.684
Recall  0.834
F1 Score  0.752


In [37]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 100, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 100, 100, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(32.9190)
5 tensor(10.2902)
10 tensor(4.4885)
15 tensor(2.1658)
20 tensor(0.6366)
25 tensor(1.6296)
30 tensor(0.9962)
35 tensor(0.7063)
40 tensor(2.2302)
45 tensor(0.6603)
50 tensor(0.8182)
55 tensor(0.8327)
60 tensor(0.6313)
65 tensor(0.9747)
70 tensor(0.6925)
75 tensor(0.6602)
80 tensor(0.4182)
85 tensor(0.6089)
90 tensor(0.4959)
95 tensor(0.6713)
100 tensor(0.5984)
105 tensor(0.4131)
110 tensor(0.4514)
115 tensor(0.4854)
120 tensor(0.4895)
125 tensor(0.4832)
130 tensor(0.7894)
135 tensor(0.5100)
140 tensor(0.7463)
145 tensor(0.3026)
150 tensor(0.5753)
155 tensor(0.3720)
160 tensor(0.4754)
165 tensor(0.3958)
170 tensor(0.4785)
175 tensor(0.4767)
180 tensor(0.3513)
185 tensor(0.4023)
190 tensor(0.3148)
195 tensor(0.4261)
Accuracy  0.821
Precision  0.778
Recall  0.85
F1 Score  0.813


In [38]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 200, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 200, 200, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:100]
    batch_y = y[:100]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(43.5609)
5 tensor(8.1452)
10 tensor(2.6511)
15 tensor(2.9570)
20 tensor(1.2538)
25 tensor(0.9190)
30 tensor(2.1377)
35 tensor(1.0743)
40 tensor(1.0297)
45 tensor(0.8179)
50 tensor(1.1411)
55 tensor(1.3800)
60 tensor(0.6207)
65 tensor(0.8470)
70 tensor(0.6330)
75 tensor(0.6057)
80 tensor(0.4245)
85 tensor(0.8043)
90 tensor(0.5482)
95 tensor(0.7438)
100 tensor(0.7053)
105 tensor(0.8950)
110 tensor(0.7068)
115 tensor(0.5087)
120 tensor(0.3072)
125 tensor(0.5766)
130 tensor(0.4090)
135 tensor(0.2918)
140 tensor(0.4522)
145 tensor(0.3297)
150 tensor(0.4041)
155 tensor(0.3641)
160 tensor(0.3352)
165 tensor(0.3146)
170 tensor(0.3617)
175 tensor(0.3731)
180 tensor(0.3673)
185 tensor(0.4902)
190 tensor(0.3299)
195 tensor(0.4324)
Accuracy  0.827
Precision  0.831
Recall  0.825
F1 Score  0.828


In [40]:
agg1 = MeanAggregator(features,n2mat)
enc1 = Encoder(features, 18, 500, ttg, agg1,n2mat)
agg2 = MeanAggregator(enc1(nodes).t(), n2mat)
enc2 = Encoder(enc1(nodes).t(), 500, 500, ttg, agg2,n2mat, base_model=enc1)

graphsage = SupervisedGraphSage(enc2)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()))
times = []
criterion = nn.CrossEntropyLoss()
for batch in range(200):
    batch_x = x[:500]
    batch_y = y[:500]
    x,y = shuffle(x,y)
    optimizer.zero_grad()
    loss = criterion(graphsage(batch_x),Variable(torch.LongTensor(batch_y)).squeeze())
    loss.backward(retain_graph=True)
    optimizer.step()
    if batch %5 == 0:
        print(batch, loss.data)

        
test_out = graphsage(xtest)
ypred = test_out.data.numpy().argmax(axis=1)
giveresults(ypred,ytest)

0 tensor(9.4583)
5 tensor(3.1506)
10 tensor(0.8428)
15 tensor(1.0863)
20 tensor(0.7788)
25 tensor(0.5713)
30 tensor(0.5374)
35 tensor(0.5545)
40 tensor(0.4626)
45 tensor(0.4704)
50 tensor(0.3696)
55 tensor(0.3887)
60 tensor(0.3847)
65 tensor(0.4363)
70 tensor(0.3797)
75 tensor(0.3391)
80 tensor(0.3500)
85 tensor(0.4251)
90 tensor(0.4280)
95 tensor(0.3946)
100 tensor(0.4747)
105 tensor(0.4105)
110 tensor(0.3686)
115 tensor(0.3567)
120 tensor(0.3820)
125 tensor(0.3520)
130 tensor(0.3531)
135 tensor(0.3655)
140 tensor(0.3870)
145 tensor(0.3676)
150 tensor(0.3689)
155 tensor(0.3427)
160 tensor(0.4912)
165 tensor(0.3560)
170 tensor(0.3372)
175 tensor(0.4955)
180 tensor(0.4041)
185 tensor(0.4140)
190 tensor(0.3305)
195 tensor(0.3169)
Accuracy  0.845
Precision  0.829
Recall  0.857
F1 Score  0.842
